# Carregamento da base

In [2]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

raw_simulated_dataset = pd.read_csv('/mnt/data/simulated_listings1_wn.csv')
raw_simulated_dataset[['latitude', 'longitude']] = raw_simulated_dataset[['latitude', 'longitude']].astype(float)
raw_simulated_dataset.drop(columns=['point'], inplace=True)
#raw_simulated_dataset.profile_report()

# Criação de features One-Hot encoding com bairros

In [3]:
dummies = pd.get_dummies(raw_simulated_dataset['neighborhood'])
raw_simulated_dataset = pd.concat([raw_simulated_dataset, dummies], axis=1).drop(columns='neighborhood')

# Separação de bases de teste e treino

In [6]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split, RepeatedKFold, cross_validate
X = raw_simulated_dataset.drop(['value', 'sold', 'time_on_market'], axis=1)
y = raw_simulated_dataset['value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=8)

O modelo é necessário para estimar o valor de venda do apartamento, seja ele com ou sem reforma necessária. Para isso não foi considerada a variável time_on_market por se tratar de uma variável que não necessáriamente temos para apartamentos que vamos comprar.

# Treinamento do modelo

In [7]:

from sklearn.metrics import mean_squared_error
from numpy import absolute
import xgboost as xgb

In [8]:
model=xgb.XGBRegressor(learning_rate = 0.01,
                           n_estimators  = 500,
                           max_depth     = 5,
                           eval_metric='rmsle')

## Avaliação da métrica

In [9]:
# define model evaluation method
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=8)
# evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_squared_log_error', cv=cv, n_jobs=-1)
print('Mean RMSLE: %.3f (%.3f)' % (absolute(scores.mean()), scores.std())) 

Mean RMSLE: -0.037 (0.002)


Foi escolhido como métrica mean squared logarithmic error porque essa métrica é recomendada quando os targets possuem crescimento exponencial, como por exemplo preço dos apartamentos.
Essa escolha é feita é para não penalizar grandes diferenças entre a predição e o valor real.

## Treinamento com validação cruzada

In [10]:
score = cross_validate(model, X, y, scoring='neg_mean_squared_log_error', cv=cv, n_jobs=-1, return_estimator=True)

In [11]:
test_score_abs = absolute(score['test_score'])
best_model_idx = np.where(test_score_abs == test_score_abs.min())
model_best = score['estimator'][best_model_idx[0][0]]

In [12]:
import pickle
file_name = "/mnt/data/xgb_price_regressor.pkl"

# save
pickle.dump(model_best, open(file_name, "wb"))

# load
#xgb_model_loaded = pickle.load(open(file_name, "rb"))

In [13]:
preds = model_best.predict(X_test)

In [14]:
df_preds = pd.DataFrame(y_test)
df_preds['preds'] = preds

df_preds

,value,preds
6446,544249,6.116443e+05
6393,627876,6.212068e+05
4924,2054370,2.461483e+06
6480,1004540,1.182210e+06
4455,1326930,1.032586e+06
...,...,...
885,1365730,1.764287e+06
3042,908762,9.342449e+05
530,487133,6.827591e+05
7752,3097650,3.019338e+06
